# How to Calculate the Park Score

## Read in Libraries

In [ ]:
import numpy as np
import pandas as pd
from datetime import date
from dateutil.relativedelta import relativedelta

## Read in Data from Open Data
- Open Data only lets you read 1000 rows by default. Change the limit `$limit=NUMBER` to make sure you're pulling all the data!

[Parks Inspection Program - Inspections Data](https://data.cityofnewyork.us/dataset/Parks-Inspection-Program-Inspections/yg3y-7juh/about_data)

In [ ]:
# Parks Inspection Program
PIPInspectionMain = pd.read_csv("https://data.cityofnewyork.us/resource/yg3y-7juh.csv?$limit=1000000")
PIPInspectionMain = PIPInspectionMain[PIPInspectionMain.inspectiontype == 'PIP']

[Parks Inspection Program - Feature Ratings](https://data.cityofnewyork.us/City-Government/Parks-Inspection-Program-Feature-Ratings/5mma-5n3h/about_data)

In [ ]:
PIPFeatureRatings = pd.read_csv("https://data.cityofnewyork.us/resource/5mma-5n3h.csv?$limit=3000000")

[Parks Inspection Program - Public Restroom Inspections](https://data.cityofnewyork.us/City-Government/Parks-Inspection-Program-Public-Restroom-Inspectio/mp8v-wjtf/about_data)

In [ ]:
PIP_PRRatings = pd.read_csv("https://data.cityofnewyork.us/resource/mp8v-wjtf.csv?$limit=1000000")

## Define Helper functions

In [ ]:
def normalize_weights(group: pd.core.groupby.generic.DataFrameGroupBy) -> pd.DataFrame:
    """
    This function normalizes the feature weights within an inspeection. 
    
    Parameters:
    - group (pd.core.groupby.generic.DataFrameGroupBy): a groupby group by InspectionID 
    
    Returns:
    - pandas Dataframe with a new column called normalized_weights where weights are normalized from 0 to 1
    
    Example:
    >>> sample_grouped = sample_df.groupby("InspectionID")
    >>> result = sample_grouped.apply(normalize_weights)
    """
    total_weight = group["weights"].sum()
    group["normalized_weights"] = group["weights"] / total_weight
    return group

In [ ]:
def filter_restroom_inspections(start: str, end: str, rated: pd.DataFrame, PIP_PRRatings: pd.DataFrame) -> pd.DataFrame:
    """
    This function ingests Public Restroom inspections and filters out unrated data.
    
    Parameters:
    - start (str): start date for the period during which inspections take place in YYYY-MM-DD format, ex. '2020-12-27'
    - end (str): end date for the period during which inspections take place in YYYY-MM-DD format, ex. '2023-12-27'
    - rated (pd.DataFrame): pd.DataFrame containing inspections 
        - contains columns: feature_1, prop_id, inspection_id, rating, date
    - PIP_PRRatings (pd.DataFrame): pd.DataFrame containing public restroom inspections 
        - contains columns: inspectionid, csnumber, cs_overall_condition, cs_litter, cs_graffiti, cs_amenities, cs_structural
    
    Returns:
    - pandas Dataframe with filtered restroom inspections
    
    Example:
    >>> result = filter_restroom_inspections("2020-12-27", "2023-12-27", rated, PIP_PRRatings)
    """
    # Take out the ratings that are meant to be closed
    restroom_inspections = PIP_PRRatings[PIP_PRRatings["cs_overall_condition"] != "N"]

    # Take out the ratings for restrooms that were supposed to be open but were not rated because they're closed
    restroom_inspections = restroom_inspections[~((restroom_inspections["cs_overall_condition"] == "U") & 
                                                  (restroom_inspections["cs_litter"] == "N") & 
                                                  (restroom_inspections["cs_graffiti"] == "N") &
                                                  (restroom_inspections["cs_amenities"] == "N") & 
                                                  (restroom_inspections["cs_structural"] == "N"))]
    
    # Take out the ratings where features are not rated, but overall condition is an A
    restroom_inspections = restroom_inspections[~((restroom_inspections["cs_overall_condition"] == "A") & 
                                                  (restroom_inspections["cs_litter"] == "N") & 
                                                  (restroom_inspections["cs_graffiti"] == "N") &
                                                  (restroom_inspections["cs_amenities"] == "N") & 
                                                  (restroom_inspections["cs_structural"] == "N"))]

    # Drop overall condition columns and clean up df
    restroom_inspections = restroom_inspections.drop(["cs_overall_condition"], axis = 1)
    restroom_inspections = restroom_inspections.set_index(["inspectionid", "csnumber"]).stack().reset_index()
    restroom_inspections = restroom_inspections.rename(columns = {"level_2": "feature", 0: "rating", "inspectionid": "inspection_id"})
    restroom_inspections["feature"] = restroom_inspections["feature"].apply(lambda x: x.split("_")[0].upper() + " " + x.split("_")[1].title())

    # Find the prop ID and date for restroom inspections
    restroom_inspections = pd.merge(restroom_inspections, 
                                    PIPInspectionMain[["prop_id", "inspection_id", "date"]], on="inspection_id", how='left')
    # Filter dates
    restroom_inspections = restroom_inspections[(restroom_inspections['date'] >= start) & 
                                                (restroom_inspections['date'] <= end)]
    
    restroom_inspections = restroom_inspections[restroom_inspections['rating'] != "N"]
    
    return restroom_inspections

In [ ]:
def apply_penalties(df: pd.DataFrame) -> pd.DataFrame:
    """
    This function takes in a filtered rated pd.DataFrame (either for cleanliness, structural, or landscape data) and
    calculates Park Score based on the weights
    
    Parameters:
    - df (pd.DataFrame): contains rated features plus their weights
    
    Returns:
    - pandas Dataframe with Park Score 
    
    Example:
    >>> df = apply_penalties(df)
    """
    df = df.groupby("inspection_id")
    df = df.apply(normalize_weights)
    df = df.drop('weights', axis = 1)
    df = df.rename(columns = {"normalized_weights": 'weights'})
    df.reset_index(drop=True, inplace=True)
    df_U_penalty = df[df["rating"] == "U"].groupby(["prop_id", "inspection_id"]).agg(u_penalty=("weights", lambda x: (x * 2).sum())).reset_index()
    df_US_penalty = df[df["rating"] == "U/S"].groupby(["prop_id", "inspection_id"]).agg(us_penalty=("weights", lambda x: (x * 6).sum())).reset_index()
    df["park_score"] = 1
    df = df.drop_duplicates(["prop_id", "inspection_id", "date"])[["prop_id", "inspection_id", "date", "park_score"]]
    df = pd.merge(df, df_U_penalty, how="left", on = ["prop_id", "inspection_id"])
    df = pd.merge(df, df_US_penalty, how="left", on = ["prop_id", "inspection_id"])
    df = df.fillna(0)
    # Floors the score at 0
    df["park_score"] = (df["park_score"] - df["u_penalty"] - df["us_penalty"]).clip(lower = 0)
    return df

In [ ]:
def calculate_cleanliness_score(cleanliness: pd.DataFrame, 
                                cleanliness_weights: dict, 
                                cleanliness_no_cs_weights: dict,
                                cs_number_per_insp_sum: pd.core.series.Series) -> pd.DataFrame:
    """
    This function takes in the rated DataFrame and returns a score based solely on cleanliness features
    
    Parameters:
    - cleanliness (pd.DataFrame): Contains cleanliness features with all their ratings 
        - contains columns: prop_id, inspection_id, feature, rating, date, inspection_type, csnumber
    - cleanliness_weights (dict): dict defining cleanliness features and weights
    - cs_number_per_insp_sum (pandas.core.series.Series): the number of public restrooms per inspection (structural includes CS Structural as a feature) 
    
    Returns:
    - pandas Dataframe with Park Score: contains columns prop_id, inspection_id, date, park_score, f_type
    
    Example:
    >>> df = calculate_cleanliness_score(df, {'Glass': 0.15, 'Graffiti': 0.05, ... }, cs_number_per_insp_sum)
    """
    
    # Separate inspections with 2 CS being inspected - at least 4 restrooms
    two_cs = cleanliness[cleanliness["inspection_id"].isin(cs_number_per_insp_sum[cs_number_per_insp_sum == 3].index)]
    two_cs["weights"] = two_cs["feature"].map(cleanliness_weights)
    # Each CS contributes to the overall restroom score equally
    two_cs.loc[two_cs["feature"].isin(["CS Litter", "CS Graffiti", "CS Amenities"]), "weights"] = two_cs[two_cs["feature"].isin(["CS Litter", "CS Graffiti", "CS Amenities"])]["weights"]/2
    two_cs_ratings = apply_penalties(two_cs)
   
    # Contains 1 CS being inspected 
    one_no_cs = cleanliness[~cleanliness["inspection_id"].isin(cs_number_per_insp_sum[cs_number_per_insp_sum == 3].index)]
    one_cs = one_no_cs[one_no_cs["inspection_id"].isin(set(one_no_cs[one_no_cs["csnumber"].notna()]["inspection_id"]))]
    one_cs["weights"] = one_cs["feature"].map(cleanliness_weights)
    one_cs_ratings = apply_penalties(one_cs)
    
    # No CS inspected contains different weights 
    no_cs = one_no_cs[~one_no_cs["inspection_id"].isin(set(one_no_cs[one_no_cs["csnumber"].notna()]["inspection_id"]))]
    no_cs["weights"] = no_cs["feature"].map(cleanliness_no_cs_weights)
    no_cs_ratings = apply_penalties(no_cs)

    cleanliness = pd.concat([no_cs_ratings, one_cs_ratings, two_cs_ratings])[["prop_id", "inspection_id", "date", "park_score"]]
    cleanliness['f_type'] = 'Cleanliness Only'
    
    return cleanliness

In [ ]:
def calculate_structural_score(rated: pd.DataFrame, structural_weights: dict, cs_number_per_insp_sum: pd.core.series.Series) -> pd.DataFrame:
    """
    This function takes in the rated DataFrame and returns a score based solely on structural features, 
    with CS Structural from public restroom inspections
    
    Parameters:
    - rated (pd.DataFrame): Contains structural features with all their ratings 
        - contains columns: prop_id, inspection_id, feature, rating, date, inspection_type, csnumber
    - structural_weights (dict): dict defining structural features and weights
    - cs_number_per_insp_sum (pandas.core.series.Series): the number of public restrooms per inspection 
    
    Returns:
    - pandas Dataframe with Park Score
    
    Example:
    >>> df = calculate_structural_score(df, {'Play Equipment': 0.25, 'Paved Surfaces': 0.25, 'Safety Surface': 0.15,...}, cs_number_per_insp_sum)
    """
    feature_list = list(structural_weights.keys())
    structural = rated[rated['feature'].isin(feature_list)]
    structural['weights'] = structural['feature'].map(structural_weights)
    # Make sure restrooms with two CS are divided by two (spread between the 0.05 for CS Structural) 
    cs_number_per_insp_sum = structural.drop_duplicates(["inspection_id", "csnumber"]).groupby(["inspection_id"])["csnumber"].sum()
    structural.loc[(structural["inspection_id"].isin(cs_number_per_insp_sum[cs_number_per_insp_sum == 3].index)) & (structural["feature"].isin(["CS Structural"])), "weights"] = structural[(structural["inspection_id"].isin(cs_number_per_insp_sum[cs_number_per_insp_sum == 3].index)) & (structural["feature"].isin(["CS Structural"]))]["weights"]/2    
    structural_ratings = apply_penalties(structural)
    structural = structural_ratings[["prop_id", "inspection_id", "date", "park_score"]].drop_duplicates()
    structural['f_type'] = 'Structural Only'
    
    return structural

In [ ]:
def calculate_landscape_score(rated: pd.DataFrame, landscape_weights: dict) -> pd.DataFrame:
    """
    This function takes in the rated DataFrame and returns a score based solely on landscape features
    
    Parameters:
    - rated (pd.DataFrame): Contains landscape features with all their ratings
    - landscape_weights (dict): dict defining landscape features and weights
    
    Returns:
    - pandas Dataframe with Park Score
    
    Example:
    >>> df = calculate_landscape_score(df, {'Lawns': 0.4,'Trees': 0.25,...})
    """
    feature_list = list(landscape_weights.keys())
    landscape = rated[rated['feature'].isin(feature_list)]
    landscape['weights'] = landscape['feature'].map(landscape_weights)
    landscape_ratings = apply_penalties(landscape)
    landscape = landscape_ratings[["prop_id", "inspection_id", "date", "park_score"]].drop_duplicates()
    landscape['f_type'] = 'Landscape Only'
    
    return landscape

In [ ]:
def calculate_weighted_score(final: pd.DataFrame, weights_dict: dict) -> pd.DataFrame:
    """
    This function takes in the rated DataFrame and returns a score based solely on cleanliness features
    
    Parameters:
    - final (pd.DataFrame): contains inspections with their park score and feature type (ex. Cleanliness, Structural, Landscape)
        - contains columns: prop_id, inspection_id, date, park_score, f_type
    
    Returns:
    - pandas Dataframe with Park Score: dataframe with a weighted score with all features
    
    Example:
    >>> df = calculate_weighted_score(df)
    """
    final = final.reset_index(drop=True)
    
    # Calculate weighted "All" score
    weighted_score = final.copy()
    weighted_score["weight"] = 0
    weighted_score.loc[weighted_score["f_type"] == "Cleanliness Only", "weight"] = weights_dict["Cleanliness"]
    weighted_score.loc[weighted_score["f_type"] == "Structural Only", "weight"] = weights_dict["Structural"]
    weighted_score.loc[weighted_score["f_type"] == "Landscape Only", "weight"] = weights_dict["Landscape"]
    
    wm = lambda x: np.average(x, weights=weighted_score.loc[x.index, "weight"])
    weighted_score = weighted_score.groupby(["prop_id", "inspection_id", "date"]).agg(weighted_score= ("park_score", wm)).reset_index()
    weighted_score = weighted_score.rename(columns={"weighted_score": "park_score"})
    weighted_score["f_type"] = "Cleanliness + Structural + Landscape"
    
    return weighted_score

In [ ]:
def calculate_PIP_Rating(PIPInspectionMain):
    """
    This function takes in the PIP Inspections data and returns a score based on pass and fail
    
    Parameters:
    - PIPInspectionMain (pd.DataFrame): main inspection data, must contain prop_id, inspection_id, date, overall_condition
    
    Returns:
    - pandas Dataframe where the PIP Inspections Score of A (Pass) is a 1 and a U (Fail) is a 0
    
    Example:
    >>> df = calculate_PIP_Rating(df)
    """
    
    # Add a PIP Rating score for comparison
    PIPRating = PIPInspectionMain[["prop_id", "inspection_id", "date", "overall_condition"]]
    PIPRating = PIPRating[(PIPRating['date'] >= start) & (PIPRating['date'] <= end)]
    PIPRating = PIPRating[PIPRating["overall_condition"] != "N"]
    PIPRating["park_score"] = PIPRating["overall_condition"].map({"A": 1, "U": 0})
    PIPRating["f_type"] = "Inspection Passing Rate"
    PIPRating = PIPRating.drop("overall_condition", axis=1)
    
    return PIPRating

## Define Park Score Function

In [ ]:
def calculate_park_score(start: str, end: str, 
                           cleanliness_weights: dict, 
                           cleanliness_no_cs_weights: dict,
                           structural_weights: dict,
                           landscape_weights: dict,
                           weights_dict = {"Cleanliness": 0.5, "Structural": 0.3, "Landscape": 0.2}) -> pd.DataFrame:
    """
    This function calculates the park score. It assumes global variables 
    PIPInspectionMain, PIPFeatureRatings, PIP_PRRatings which are
    the PIP Inspection overall ratings, feature ratings, and PIP Public Restroom Ratings, respectively.
    PIPInspectionMain must contain these columns: inspection_id, date of inspection, and prop_id of 
    where the inspection took place. PIPFeatureRatings must contain these columns: inspection_id, feature_1, and rating.
    PIP_PRRatings must contain these columns: inspectionid, csnumber (number of public restrooms, 
    where 1 represents a men's and women's restroom), and scores for each feature, cs_overall_condition, 
    cs_litter, cs_graffiti, cs_amenities, cs_structural.
    
    Parameters:
    - start (str): start date for inspections data ex. '2020-12-27'
    - end (str): end date for inspections data, ex. '2020-12-27'
    - cleanliness_weights (dict): defines weights for every cleanliness features including restrooms
    - cleanliness_no_cs_weights (dict): defines weights for cleanliness features without restrooms
    - structural_weights (dict): defines weights to structural features
    - landscape_weights (dict): defines weights for landscape features
    - weights_dict (dict): defines weights for overall cleanliness, structural, and landscape categories
    
    Returns:
    - pandas Dataframe with a Feature Type and overall Park Score from 0 to 100 per Inspection

    """
    
    ### Merge inspections and restroom inspections data
    # Merge feature and inspection ratings
    inspections = pd.merge(PIPFeatureRatings, PIPInspectionMain, on = "inspection_id")
    # Filter any features not rated
    rated = inspections[inspections['rating'] != "N"]
    # Select only columns needed
    rated = rated[["feature_1", "prop_id", "inspection_id", "rating", "date"]]
    rated = rated.rename(columns = {"feature_1": "feature"})
    # Filter dates
    rated = rated[(rated['date'] >= start) & (rated['date'] <= end)]

    # Clean Restroom Inspections Data
    restroom_inspections = filter_restroom_inspections(start, end, rated, PIP_PRRatings)
     # Filter out restroom inspections that aren't in the main inspections 
    restroom_inspections= restroom_inspections[~restroom_inspections["inspection_id"].isin(set(restroom_inspections["inspection_id"]) - set(rated["inspection_id"]))]
    # Define Inspection Type
    rated["inspection_type"] = "Inspection"
    restroom_inspections["inspection_type"] = "Inspection with Restroom"
    
    # Concatenate everything
    rated = pd.concat([rated, restroom_inspections])
    rated["rating"] = rated["rating"].str.upper()
    
    # Differentiate Inspection Types
    allIDs = set(rated["inspection_id"])
    restroomIDs = set(rated[rated["inspection_type"] == "Inspection with Restroom"]["inspection_id"])
    inspectionIDs = allIDs - restroomIDs
    IDmappings = pd.DataFrame(pd.concat([pd.Series("Inspection", index=inspectionIDs), pd.Series("Inspection with Restroom", index=restroomIDs)])).reset_index()
    IDmappings = IDmappings.rename(columns = {"index": "inspection_id", 0: "inspection_type"})
    
    ### Calculate Cleanliness, Structural, and Landscape Scores
    # create table of ratings based only on cleanliness, then only structural, then only landscape
    cleanliness = rated[rated["feature"].isin(list(cleanliness_weights.keys()))]
    cs_number_per_insp_sum = cleanliness.drop_duplicates(["inspection_id", "csnumber"]).groupby(["inspection_id"])["csnumber"].sum()
    
    cleanliness = calculate_cleanliness_score(cleanliness, cleanliness_weights, cleanliness_no_cs_weights, cs_number_per_insp_sum)
    structural = calculate_structural_score(rated, structural_weights, cs_number_per_insp_sum)
    landscape = calculate_landscape_score(rated, landscape_weights)
    
    
    ### Concatenate 3 feature type tables with overall calculated score
    final = pd.concat([cleanliness, structural, landscape])
    weighted_score = calculate_weighted_score(final, weights_dict)
    PIPRating = calculate_PIP_Rating(PIPInspectionMain)
    final = pd.concat([final, weighted_score, PIPRating])
    final = final.reset_index(drop=True)
    final["park_score"] = final["park_score"] * 100
    
   
    final = pd.merge(final, IDmappings, how="left", on = "inspection_id")

    return final

## Calculate Park Score

### This grabs the scores of all parks starting from today, covering the past 3 years

In [ ]:
today = date.today()
rolling_period_year = 3

end = (today).strftime("%Y-%m-%d")
start = (today - relativedelta(years = rolling_period_year)).strftime("%Y-%m-%d")

In [ ]:
# Define the weights for different categories
# ice and weeds should switch off and there should never be a ice AND weeds rating
cleanliness_weights = {"Glass": 0.15, "Graffiti": 0.05, "Ice": 0.05, "Weeds": 0.05, "Litter": 0.30, 
                       "CS Litter": 0.25, "CS Graffiti": 0.05, "CS Amenities": 0.15}
# If no CS, it's exactly the same weights for original Cleanliness features 
cleanliness_no_cs_weights = {"Glass": 15/55, "Graffiti": 5/55, "Ice": 5/55, "Weeds": 5/55, "Litter": 30/55}

structural_weights = {'Play Equipment': 0.25, 'Paved Surfaces': 0.25, 'Safety Surface': 0.15, 
                      'Benches': 0.15, 'Sidewalks': 0.10, 'CS Structural': 0.05, 'Fences': 0.05}
landscape_weights = {"Lawns": 0.40, 'Trees': 0.25, 'Athletic Fields': 0.20, 
                     'Horticultural Areas': 0.05, 'Water Bodies': 0.05, 'Trails': 0.05}



In [ ]:
park_score = calculate_park_score(start, end, cleanliness_weights, cleanliness_no_cs_weights, structural_weights, landscape_weights)

In [ ]:
park_score.head()

## Aggregate to a Park Level from Zone Level

[Parks Inspection Program – All Sites (MAPPED)](https://data.cityofnewyork.us/City-Government/Parks-Inspection-Program-All-Sites-MAPPED-/buk3-3qpr/about_data)

In [ ]:
AllSites = pd.read_csv("https://data.cityofnewyork.us/resource/buk3-3qpr.csv?$limit=3000000")

In [ ]:
# Merge with Park Score to retrieve Parks (PropNum)
park_score = pd.merge(park_score, AllSites[["prop_name", "prop_id", "propnum"]].drop_duplicates(), on='prop_id', how='left')

In [ ]:
# Retrieve the score for all
park_score_all = park_score[park_score["f_type"] == "Cleanliness + Structural + Landscape"]

In [ ]:
park_score_all.groupby(["propnum", "prop_name"])["park_score"].mean()